# IoT Examples (Tested on Wrangler/Jetstream)


    VM Firewall Rules
    
    To                         Action      From
    --                         ------      ----
    22/tcp                     ALLOW       Anywhere
    Anywhere                   ALLOW       129.114.63.32/27
    Anywhere                   ALLOW       129.114.17.136
    8888                       ALLOW       Anywhere
    Anywhere                   ALLOW       206.76.196.32/27
    Anywhere                   ALLOW       206.76.0.0/16
    22/tcp (v6)                ALLOW       Anywhere (v6)
    8888 (v6)                  ALLOW       Anywhere (v6)

In [ ]:
import os, sys
import distributed
import json
import pilot.streaming
from confluent_kafka import Producer, Consumer
import socket

sys.path.append("../../")
# configure logging
import logging

#logging.getLogger().setLevel(logging.DEBUG)
logging.getLogger("stevedore.extension").setLevel(logging.CRITICAL)
logging.getLogger("keystoneauth").setLevel(logging.CRITICAL)
logging.getLogger("urllib3.connectionpool").setLevel(logging.CRITICAL)
logging.getLogger("asyncio").setLevel(logging.CRITICAL)
sys.modules['pilot.streaming']


# Configuration
RESOURCE_URL_DASK = "ssh://aluckow@js-129-114-17-61.jetstream-cloud.org"
RESOURCE_URL_KAFKA = "ssh://aluckow@js-129-114-17-61.jetstream-cloud.org"

WORKING_DIRECTORY = os.path.join(os.environ["HOME"], "work")

# RESOURCE_URL_EDGE="os://cc.lrz.de"
# RESOURCE_URL_EDGE="ssh://localhost"

# Start Kafka

In [ ]:
pilot_compute_description = {
    "resource": RESOURCE_URL_KAFKA,
    "working_directory": WORKING_DIRECTORY,
    "number_of_nodes": 1,
    "cores_per_node": 1,
    "config_name": "jetstream",
    "walltime": 59,
    "type": "kafka"
}
kafka_pilot = pilot.streaming.PilotComputeService.create_pilot(pilot_compute_description)
kafka_pilot.wait()

In [ ]:
kafka_details=kafka_pilot.get_details()
kafka_details

In [ ]:
topic_name = "pilot-streaming-test"

conf = {'bootstrap.servers': kafka_details["bootstrap_servers"],
        'group.id': "radical2",
        'auto.offset.reset': 'earliest'}

producer = Producer(conf)
consumer = Consumer(conf)
consumer.subscribe([topic_name])

producer.produce(topic_name, key="key", value="Hello World")
producer.flush()

In [ ]:
msg = consumer.poll(timeout=2.0)

if msg is not None:
    print(msg.value())